In [1]:
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import mani_skill.envs
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from mani_skill.utils import gym_utils
from mani_skill.utils.wrappers.record import RecordEpisode
from mani_skill.vector.wrappers.sb3 import ManiSkillSB3VectorEnv

ms3_vec_env = gym.make("PegInsertionSide-v1", num_envs=4096, control_mode='pd_joint_delta_pos')
vec_env = ManiSkillSB3VectorEnv(ms3_vec_env)

#Define PPO Agent and train
model = PPO("MlpPolicy", vec_env, gamma=0.8, gae_lambda=0.9, n_steps=50, batch_size=256, n_epochs=8, verbose=1)
model.learn(total_timesteps=200_000_000)
model.save("ppo_peginsert_200M")



/home/tkj9ep/.conda/envs/maniskill_env/lib/python3.9/site-packages/torch/random.py:167: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(message)
/home/tkj9ep/.conda/envs/maniskill_env/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to ge

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 1.07     |
| time/              |          |
|    fps             | 8985     |
|    iterations      | 1        |
|    time_elapsed    | 22       |
|    total_timesteps | 204800   |
---------------------------------


KeyboardInterrupt: 

In [3]:
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import mani_skill.envs
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from mani_skill.utils import gym_utils
from mani_skill.utils.wrappers.record import RecordEpisode
from mani_skill.vector.wrappers.sb3 import ManiSkillSB3VectorEnv
import torch
from PIL import Image
import os
import shutil




##############################################################################################

num_envs = 4
max_episode_steps = 75
success = torch.zeros(num_envs, dtype=torch.bool).cuda()
model = PPO.load('ppo_peginsert_200M')
#Visualize the trained agent
eval_vec_env = gym.make("PegInsertionSide-v1", num_envs=num_envs, control_mode='pd_joint_delta_pos', render_mode="rgb_array", sim_backend='gpu')
eval_vec_env = RecordEpisode(eval_vec_env, output_dir="Videos", save_video=True, save_trajectory=False, max_steps_per_video=max_episode_steps)
obs, _ = eval_vec_env.reset()

for i in range(max_episode_steps):
    action, _states = model.predict(obs.cpu().numpy(), deterministic=True)
    obs, rewards, dones, _, info = eval_vec_env.step(action)
    print(info['success'])
    # Collect success info
    step_success = info['success']  # This is a tensor
    success = torch.logical_or(success, step_success)  # Accumulate success across steps
# Count the number of successful environments
success_count = torch.sum(success).item()
success_rate = success_count / num_envs


# Save video with a better name than 0
video_dir = "Videos"
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]


if len(video_files) > 0:
    # Get the most recent video file and rename it
    latest_video = max(video_files, key=lambda f: os.path.getctime(os.path.join(video_dir, f)))
    # Create a unique name for each task video
    new_video_name = f"{'peginsert'}.mp4"

    # Rename the video file to avoid overwriting
    shutil.move(os.path.join(video_dir, latest_video), os.path.join(video_dir, new_video_name))
    print(f"Video saved and renamed to: {new_video_name}")
else:
    print("No video files found to rename.")

#Save image
image_tensor = eval_vec_env.render()[0] # torch tensor image as 256 x 256 x 3
# Convert torch tensor to PIL Image
image = Image.fromarray(image_tensor.cpu().numpy().astype('uint8'))
image.save("peginsert.jpg")
# Print success data
print(f"Number of successful episodes: {success_count}")
print(f"PullCube Success rate: {success_rate:.2%}")



tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False,

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

# free up GPU memory
torch.cuda.empty_cache()

# Load the image
image = Image.open("pickcube.jpg")

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "qresearch/llama-3.1-8B-vision-378",  # The pre-trained Vision-Language Model (VLM)
    trust_remote_code=True,
    torch_dtype=torch.float16  # Use FP16 for better memory efficiency
).to("cuda")  # Move the model to the GPU

tokenizer = AutoTokenizer.from_pretrained(
    "qresearch/llama-3.1-8B-vision-378",  # Same model for the tokenizer
    use_fast=True
)

# Ask question
question = "Describe the scene in detail."

# Generate the description
with torch.no_grad():  # Disable gradients to save memory
    description = model.answer_question(
        image, question, tokenizer,
        max_new_tokens=128,  # Max tokens for the answer
        do_sample=True,      # Sampling to get diverse results
        temperature=0.3      # Control randomness
    )

# Print the generated description
print("Image description:", description)


In [3]:
torch.cuda.empty_cache()